In [0]:
#!pip install category_encoders

In [0]:
import pandas as pd
import numpy as np
import gc
from category_encoders.ordinal import OrdinalEncoder
from tqdm import tqdm
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [3]:
%%time
data = pd.read_excel('/content/drive/My Drive/Data/card transactions.xlsx', converters={'Cardnum':str})

CPU times: user 9.26 s, sys: 88.4 ms, total: 9.35 s
Wall time: 9.39 s


In [4]:
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [5]:
%%time
for keys in [['Cardnum', 'Merchnum'], ['Cardnum', 'Merch zip'], ['Cardnum', 'Merch state']]:
    data['_'.join(keys)] = data[keys[0]].apply(str) + '_' + data[keys[1]].apply(str)

CPU times: user 147 ms, sys: 16 ms, total: 163 ms
Wall time: 164 ms


In [6]:
%%time
attributes = ['Cardnum', 'Merchnum', 'Cardnum_Merchnum', 'Cardnum_Merch zip', 'Cardnum_Merch state']
encoder = OrdinalEncoder()
data[attributes] = encoder.fit_transform(data[attributes])

CPU times: user 389 ms, sys: 16 ms, total: 405 ms
Wall time: 407 ms


In [0]:
%%time
days = ['1d', '2d', '4d', '8d', '15d', '31d']
fns = ['mean', 'max', 'median', 'sum']
for attribute in tqdm(attributes):
    for day in days:
        for fn in fns:
            data['Amount_'+attribute+'_'+fn+'_'+day] = data[['Date', attribute]].merge(data.set_index('Date').groupby(attributes)['Amount'].rolling(day).agg(fn).reset_index().drop_duplicates(subset = ['Date', attribute]), on = ['Date', attribute], how = 'left')['Amount']
            data['Actual_Amount/'+attribute+'_'+fn+'_'+day] = data['Amount'] / data['Amount_'+attribute+'_'+fn+'_'+day]
        data[attribute+'_count_'+day] = data[['Date', attribute]].merge(data.set_index('Date').groupby(attribute)[attribute].rolling(day).count().rename('temp').reset_index().drop_duplicates(subset = ['Date', attribute]), on = ['Date', attribute], how = 'left')['temp']
        data['Days_since_'+attribute] = data.groupby(attribute)['Date'].diff() / np.timedelta64(1, 'D')

  0%|          | 0/5 [00:00<?, ?it/s]

In [0]:
data[attributes] = encoder.inverse_transform(data[attributes])

In [0]:
data.head()

In [0]:
data.shape

In [0]:
data.to_csv('features.csv', index = False)